In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
import xarray as xar

C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
file_path='./data/sanjioil.nc'

In [5]:
ds=nc.Dataset(file_path)

In [6]:
ds_xr=xar.open_dataset(file_path)

# 1 按照指定时间读取对应的全部散点值

In [26]:
xr_temp=ds_xr.isel(time=1).get('lat')
xr_temp

<xarray.DataArray 'lat' (trajectory: 2958)>
array([2.836955e+01, 2.837354e+01, 2.838293e+01, ..., 9.969210e+36,
       9.969210e+36, 9.969210e+36], dtype=float32)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-14T23:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 28.369555 28.373537 ... 9.96921e+36
Attributes:
    units:          degrees_north
    standard_name:  latitude
    long_name:      latitude

ValueError: cannot create a Dataset from a DataArray with the same name as one of its coordinates


TypeError: can only lookup dictionaries from Dataset.loc

In [29]:
ds_xr.loc['lat','lon']

TypeError: can only lookup dictionaries from Dataset.loc

获取所有status符合条件的散点

In [35]:
ds_xr.isel(time=[0]).get('status').to_dataframe()['lat'].head()

trajectory  time               
1           2018-01-14 22:20:00    28.369371
2           2018-01-14 22:20:00    28.369326
3           2018-01-14 22:20:00    28.369295
4           2018-01-14 22:20:00    28.369312
5           2018-01-14 22:20:00    28.369246
Name: lat, dtype: float32

In [36]:
ds_xr.isel(time=[0]).get('status').to_dataframe().head()

,,lon,lat,status
trajectory,time,,,
1,2018-01-14 22:20:00,125.921562,28.369371,0
2,2018-01-14 22:20:00,125.921494,28.369326,0
3,2018-01-14 22:20:00,125.921516,28.369295,0
4,2018-01-14 22:20:00,125.921585,28.369312,0
5,2018-01-14 22:20:00,125.921562,28.369246,0


In [42]:
type(ds_xr.isel(time=[0]).get('status'))

xarray.core.dataarray.DataArray

# 2 将`DataArray` 转成 `DataSet`

In [44]:
xr_merge=xar.merge([ds_xr.isel(time=[0]).get('status')])

In [45]:
xr_merge

<xarray.Dataset>
Dimensions:     (time: 1, trajectory: 2958)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
  * time        (time) datetime64[ns] 2018-01-14T22:20:00
    lon         (trajectory, time) float32 ...
    lat         (trajectory, time) float32 ...
Data variables:
    status      (trajectory, time) int32 ...

In [48]:
ds_xr.isel(time=[0]).get('lat')

<xarray.DataArray 'lat' (trajectory: 2958, time: 1)>
array([[2.836937e+01],
       [2.836933e+01],
       [2.836930e+01],
       ...,
       [9.969210e+36],
       [9.969210e+36],
       [9.969210e+36]], dtype=float32)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
  * time        (time) datetime64[ns] 2018-01-14T22:20:00
    lon         (trajectory, time) float32 ...
    lat         (trajectory, time) float32 28.369371 28.369326 ... 9.96921e+36
Attributes:
    units:          degrees_north
    standard_name:  latitude
    long_name:      latitude

In [47]:
xr_merge=xar.merge([ds_xr.isel(time=[0]).get('lat')])

ValueError: cannot create a Dataset from a DataArray with the same name as one of its coordinates

In [50]:
xr_merge=xr_merge.where(xr_merge>=0).where(xr_merge<1).to_dataframe().dropna(how='any')
xr_merge

status         lon        lat
time                trajectory                               
2018-01-14 22:20:00 1              0.0  125.921562  28.369371
                    2              0.0  125.921494  28.369326
                    3              0.0  125.921516  28.369295
                    4              0.0  125.921585  28.369312
                    5              0.0  125.921562  28.369246
                    6              0.0  125.921417  28.369282
                    7              0.0  125.921516  28.369305
                    8              0.0  125.921463  28.369305
                    9              0.0  125.921463  28.369230
                    10             0.0  125.921494  28.369314
                    11             0.0  125.921478  28.369312
                    12             0.0  125.921547  28.369284
                    13             0.0  125.921509  28.369301
                    14             0.0  125.921478  28.369232
                    15             0.0  125.921494  28.369234
                    16             0.0  125.921486  28.369307
                    17             0.0  125.921547  28.369270
                    18             0.0  125.921463  28.369299
                    19             0.0  125.921486  28.369337
                    20             0.0  125.921425  28.369291
                    21             0.0  125.921341  28.369173

获取新生成的df的长度

In [59]:
len(xr_merge)

21

In [58]:
type(xr_merge.iloc[0])

pandas.core.series.Series

In [52]:
xr_merge.iloc[0]

status      0.000000
lon       125.921562
lat        28.369371
Name: (2018-01-14 22:20:00, 1), dtype: float64

In [57]:
xr_merge.iloc[0]['status']

0.0

In [55]:
xr_merge.iloc[0]['lat']

28.36937141418457

* 此处有一个问题，如果获取name:直接 .name即可  
* 将`pandas._libs.tslibs.timestamps.Timestamp` -> `datetime`

In [60]:
xr_merge.iloc[0].name

(Timestamp('2018-01-14 22:20:00'), 1)

In [61]:
type(xr_merge.iloc[0].name)

tuple

In [62]:
xr_merge.iloc[0].name[0]

Timestamp('2018-01-14 22:20:00')

In [63]:
type(xr_merge.iloc[0].name[0])

pandas._libs.tslibs.timestamps.Timestamp

In [64]:
xr_merge.iloc[0].name[0].to_datetime()

AttributeError: 'Timestamp' object has no attribute 'to_datetime'

In [66]:
pd.to_datetime(xr_merge.iloc[0].name[0])

Timestamp('2018-01-14 22:20:00')

In [67]:
xr_merge.iloc[0].name[0].to_pydatetime()

datetime.datetime(2018, 1, 14, 22, 20)